# CIFAR-10 Inference pipeline
Author: **Andrea Incerti Delmonte**

Email: ** andrea.incertidelmonte@gmail.com**

This inference pipeline is based on TF Estimator https://www.tensorflow.org/get_started/custom_estimators
1. Load CIFAR-10 metadata
2. Load CIFAR-10 test data
3. Load exported predictor
4. Apply inference on test data

In [ ]:
import tensorflow as tf
import os
import cPickle
import numpy as np
import matplotlib.pyplot as plt
# Visualizations will be shown in the notebook.
%matplotlib inline
import random

In [ ]:
CIFAR10_LOCAL_FOLDER = "./cifar-10_dataset/cifar-10-batches-py"
IMG_HEIGHT = 32
IMG_WIDTH = 32
IMG_CHANNELS = 3

###############################
#                             #
# Choose which model to train #
#                             #
###############################

# LeNet
TRAINED_MODEL_EXPORT_DIR = "./trained_models/lenet/export/exported-lenet"

# DeepLeNet
#TRAINED_MODEL_EXPORT_DIR = "./trained_models/deep-lenet/export/exported-deep-lenet"

## 1. Load CIFAR-10 metadata

In [ ]:
metadata_f = open(os.path.join(CIFAR10_LOCAL_FOLDER, "batches.meta"), "rb")
metadata_dict = cPickle.load(metadata_f)
labels_LUT = metadata_dict["label_names"]
for index, value in enumerate(labels_LUT):
    print("Label {} = {}".format(index,value))

## 2. Load CIFAR-10 test data

In [ ]:
test_f = open(os.path.join(CIFAR10_LOCAL_FOLDER, "test_batch"), "rb")
test_dict = cPickle.load(test_f)
test_images = test_dict['data']
test_labels = np.asarray(test_dict['labels'])
    
print("test_images.shape {}".format(test_images.shape))
print("test_labels.shape {}".format(test_labels.shape))

## 3. Load exported predictor

In [ ]:
# Pick last exported version of the model
saved_model_dir = os.path.join(TRAINED_MODEL_EXPORT_DIR, os.listdir(TRAINED_MODEL_EXPORT_DIR)[-1]) 

predictor_fn = tf.contrib.predictor.from_saved_model(
    export_dir = saved_model_dir,
    signature_def_key='predictions'
)

## 4. Apply inference on test data

In [ ]:
def plot_image(image, label, predicted_label):
    image = image
    plt.imshow(image)
    plt.title("Image index {}, label {}, predicted label {}".format(index, labels_LUT[label], labels_LUT[predicted_label]))
    plt.show()

### 4.1 Plot wrong predictions

In [ ]:
for i in range(10):
    index = random.randint(0, test_images.shape[0])
    
    image = test_images[index].reshape([1, IMG_CHANNELS, IMG_HEIGHT, IMG_WIDTH]).transpose([0, 2, 3, 1])
    label = test_labels[index]
    
    output = predictor_fn({'images': image})
    predicted_label = np.argmax(output['probabilities'])
    
    if predicted_label != label:
        print(index)
        print(label, labels_LUT[label])
        print(predicted_label, labels_LUT[predicted_label])
        plot_image(image[0], label, predicted_label)    

### 4.1 Compute accuracy

In [ ]:
samples = 1000

output = predictor_fn({
    'images': test_images[:samples].reshape([samples, IMG_CHANNELS, IMG_HEIGHT, IMG_WIDTH]).transpose([0, 2, 3, 1])
})

correct_predictions = np.sum([label==predicted_label for label, predicted_label in zip(test_labels, output['class_ids'])])
accuracy = correct_predictions / float(samples)

print("Accuracy on {} test samples: {}".format(samples, accuracy))